# Living Vendor Generator 物资供应商

##  爬虫

#### 依赖

In [94]:
!python3 -m pip install -r requirements.txt

  Using cached pysqlite3-0.4.7.tar.gz (40 kB)
  Preparing metadata (setup.py) ... done
  Command arguments: /Library/Frameworks/Python.framework/Versions/3.7/bin/python3 -u -c '
  exec(compile('"'"''"'"''"'"'
  # This is <pip-setuptools-caller> -- a caller that pip uses to run setup.py
  #
  # - It imports setuptools before invoking setup.py, to enable projects that directly
  #   import from `distutils.core` to work with newer packaging standards.
  # - It provides a clear error message when setuptools is not installed.
  # - It sets `sys.argv[0]` to the underlying `setup.py`, when invoking `setup.py` so
  #   setuptools doesn'"'"'t think the script is `-c`. This avoids the following warning:
  #     manifest_maker: standard file '"'"'-c'"'"' not found".
  # - It generates a shim setup.py, for handling setup.cfg-only projects.
  import os, sys, tokenize
  
  try:
      import setuptools
  except ImportError as error:
      print(
          "ERROR: Can not execute `setup.py` since setu

In [11]:
import os
import sys
import logging
import numpy as np
import pandas as pd

FORMAT = '%(levelname)s %(asctime)s [%(filename)s:%(lineno)d]: %(message)s'
logging.basicConfig(format=FORMAT, stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()

#### 建立scrapy工程

In [10]:
!scrapy startproject living_manual

New Scrapy project 'living_manual', using template directory '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/yixiaobin/local/workspace/living_manual_at_2022/living_manual

You can start your first spider with:
    cd living_manual
    scrapy genspider example example.com


#### 运行scrapty

In [11]:
!cd crawler; scrapy crawl living-spider

2022-04-06 16:49:33 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: crawler)
2022-04-06 16:49:33 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 16:52:21) - [Clang 6.0 (clang-600.0.57)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.2, Platform Darwin-21.3.0-x86_64-i386-64bit
2022-04-06 16:49:33 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawler',
 'NEWSPIDER_MODULE': 'crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['crawler.spiders']}
2022-04-06 16:49:33 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-04-06 16:49:33 [scrapy.extensions.telnet] INFO: Telnet Password: 30df4b8bb8c00aee
2022-04-06 16:49:33 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsa

#### 使用sqlite

In [3]:
import os
import sqlite3

class DB:
    def __init__(self):
        self.db_con = sqlite3.connect(os.path.join('db', 'living_vendor', 'living_vendor.db'))
        

    def find(self, table_name):
        return self._select_(f'select * from {table_name}')

    def get(self, table_name, id):
        statement = (f'select * from {table_name} where id = ?', (id,))
        dataset = self._select_(statement)
        if len(dataset) == 1:
            return dataset[0]
        return None

    def query(self, sql):
        return self._select_(sql)

    def create(self, table_name, **scheme):
        sqls = []
        sqls.append(f'DROP TABLE IF EXISTS {table_name}')
        fields = []
        for n, t in scheme.items():
            fields.append(f'{n} t')
        fields = ','.join(fields)
        sqls.append(f'create table {table_name}({fields})')
        self._udpate_(sqls)

    def insert(self, table_name, **fields):
        names = []
        values = []
        for k, v in fields.items():
            names.append(k)
            values.append(v)
        names = ','.join(names)
        if len(fields.items()) == 1:
            refs = '?'
        else:
            refs = ','.join('?' for _ in range(len(fields.items())))
        sql = f'insert into {table_name}({names}) values({refs})'
        statement = (sql, values)
        self._udpate_([statement])

    def delete(self, table_name):
        sql = f'delete from {table_name}'
        self._udpate_([sql])

    def drop(self, table_name):
        sql = f'drop table if exists {table_name}'
        self._udpate_([sql])

    def update(self, sql):
        self._udpate_([sql])

    def _udpate_(self, sqls):
        db_cur = self.db_con.cursor()
        for sql in sqls:
            logger.debug(sql)
            if type(sql) is tuple:
                db_cur.execute(sql[0], sql[1])
            else:
                db_cur.execute(sql)
        self.db_con.commit()

    def _select_(self, sql):
        db_cur = self.db_con.cursor()
        if type(sql) is tuple:
            db_cur.execute(sql[0], sql[1])
        else:
            db_cur.execute(sql)
        dataset = db_cur.fetchall()
        db_cur.close()
        return dataset

    @staticmethod
    def update_scheme():
        os.system('alembic current')
        os.system('alembic history')
        os.system('alembic upgrade head')
        os.system('alembic current')

In [4]:
db = DB()

In [5]:
db.query('SELECT name FROM sqlite_master WHERE type="table"')

[('alembic_version',),
 ('district',),
 ('vendor',),
 ('cart',),
 ('tag',),
 ('vendor_tag',),
 ('cart_goods',),
 ('goods',),
 ('product',),
 ('category',),
 ('vendor_district',)]

In [6]:
db.query('SELECT * from district')

[(1,
  '上海市',
  '31',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (2,
  '黄浦区',
  '310101',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (3,
  '徐汇区',
  '310104',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (4,
  '长宁区',
  '310105',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (5,
  '静安区',
  '310106',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (6,
  '普陀区',
  '310107',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (7,
  '虹口区',
  '310109',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (8,
  '杨浦区',
  '310110',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (9,
  '闵行区',
  '310112',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (10,
  '宝山区',
  '310113',
  '2022-04-13 20:42:15.065849',
  '2022-04-13 20:42:15.065849',
  1),
 (11,
  '嘉定区',
  '310114',
  '2022-04-13 20

In [12]:
db.create('test', id='int')

DEBUG 2022-04-14 20:10:01,843 [<ipython-input-3-6b24944e4fdd>:61]: DROP TABLE IF EXISTS test
DEBUG 2022-04-14 20:10:01,844 [<ipython-input-3-6b24944e4fdd>:61]: create table test(id t)


In [13]:
db.query('SELECT name FROM sqlite_master WHERE type="table"')

[('alembic_version',),
 ('district',),
 ('vendor',),
 ('cart',),
 ('tag',),
 ('vendor_tag',),
 ('cart_goods',),
 ('goods',),
 ('product',),
 ('category',),
 ('vendor_district',),
 ('test',)]

In [14]:
db.query('SELECT * FROM test')

[]

In [15]:
db.insert('test', id='1')

DEBUG 2022-04-14 20:10:03,326 [<ipython-input-3-6b24944e4fdd>:61]: ('insert into test(id) values(?)', ['1'])


In [423]:
db.query('SELECT * FROM test')

[(1,)]

## 数据库

### 初始化

In [104]:
!alembic init db

  Creating directory /Users/yixiaobin/local/workspace/living_manual_at_2022/db ...  done
  Creating directory
  /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions ...  done
  Generating
  /Users/yixiaobin/local/workspace/living_manual_at_2022/db/script.py.mako ...  done
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/env.py ...  done
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/README ...  done
  File /Users/yixiaobin/local/workspace/living_manual_at_2022/alembic.ini already
  exists, skipping
  Please edit configuration/connection/logging settings in
  '/Users/yixiaobin/local/workspace/living_manual_at_2022/alembic.ini' before
  proceeding.


### 数据库设计

![living_vendor_db](./living_vendor_db.jpg)

### 建立Schema

In [59]:
class Entity:
    def __init__(self):
        self.id = -1
        self.created_at = None
        self.updated_at = None 
        self.state = 1
        
    def __repr__(self):
        return str(self.__dict__)    

#### 建立 district

In [319]:
!alembic revision -m "create district table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/5b
  1aa6c84891_create_district_table.py ...  done


In [320]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 23:20:05,474 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 23:20:05,474 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:20:05,474 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA temp.table_info("alembic_version")
2022-04-06 23:20:05,475 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:20:05,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
2022-04-06 23:20:05,475 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 2

In [321]:
db.query('SELECT name FROM sqlite_master WHERE type="table"')

[('test',),
 ('alembic_version',),
 ('district',),
 ('vendor',),
 ('cart',),
 ('tag',),
 ('vendor_tag',),
 ('cart_goods',),
 ('goods',),
 ('product',),
 ('category',)]

In [322]:
db.query('select * from alembic_version')

[('5b1aa6c84891',)]

#### 插入 district

In [258]:
!alembic revision -m "insert district table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/e3
  e84b19051d_insert_district_table.py ...  done


DEBUG 2022-04-06 22:44:12,591 [<ipython-input-214-269850c89446>:49]: drop table if exists alembic_version


In [259]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 22:41:29,324 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 22:41:29,324 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 22:41:29,326 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 22:41:29,326 INFO sqlalchemy.engine.Engine [generated in 0.00032s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00032s] ()
2022-04-06 22:41:29,329 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade eb7ebea08315 -> e3e84b19051d, insert district table
2022-04-06 22:41:29,331

In [284]:
class District(Entity):
    def __init__(self):
        super().__init__()
        self.name = ''
        self.code = ''

#### 建立vendor

In [286]:
!alembic revision -m "create vendor table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/8f
  b6c7a14981_insert_vendor_table.py ...  done


In [289]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 22:51:47,925 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 22:51:47,925 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 22:51:47,926 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 22:51:47,926 INFO sqlalchemy.engine.Engine [generated in 0.00014s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00014s] ()
2022-04-06 22:51:47,930 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade eb7ebea08315 -> 8fb6c7a14981, insert vendor table
2022-04-06 22:51:47,931 I

In [78]:
class Vendor(Entity):
    def __init__(self):
        super().__init__()
        self.name = ''
        self.contact = ''
        self.address = ''
        self.mobile = ''
        self.tel = ''
        self.desc = ''
        self.wechat = ''
        self.origin = ''
        self.available_desc = ''

#### 建立tag

In [296]:
!alembic revision -m "create tag table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/a0
  b4495ce7fb_insert_tag_table.py ...  done


In [297]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 22:56:18,250 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 22:56:18,250 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 22:56:18,251 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 22:56:18,252 INFO sqlalchemy.engine.Engine [generated in 0.00017s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00017s] ()
2022-04-06 22:56:18,255 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade fab52715b8d1 -> a0b4495ce7fb, insert tag table
2022-04-06 22:56:18,257 INFO

In [61]:
class Tag(Entity):
    def __init__(self):
        super().__init__()
        self.name = ''

### 建立vendor_tag

In [300]:
!alembic revision -m "create vendor_tag table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/10
  ef925b9a32_insert_vendor_tag_table.py ...  done


In [391]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-07 00:22:56,751 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-07 00:22:56,751 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-07 00:22:56,752 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-07 00:22:56,753 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-07 00:22:56,753 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
2022-04-07 00:22:56,753 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engi

In [392]:
class VendorTag(Entity):
    def __init__(self):
        super().__init__()
        self.vendor_id = -1
        self.tag_id = -1

#### 建立cart

In [393]:
!alembic revision -m "create cart table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/f6
  5d0537df7b_create_cart_table.py ...  done


In [394]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-07 00:23:11,559 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-07 00:23:11,559 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-07 00:23:11,560 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-07 00:23:11,560 INFO sqlalchemy.engine.Engine [generated in 0.00014s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00014s] ()
2022-04-07 00:23:11,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 4ba3059ff18c -> f65d0537df7b, create cart table
2022-04-07 00:23:11,566 INF

In [298]:
class Cart(Entity):
    def __init__(self):
        super().__init__()
        self.vendor_id = -1

#### 建立cart_goods

In [304]:
!alembic revision -m "create cart_goods table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/3a
  1141b115bc_insert_cart_goods_table.py ...  done


In [305]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 23:01:53,120 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 23:01:53,120 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:01:53,122 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 23:01:53,122 INFO sqlalchemy.engine.Engine [generated in 0.00014s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00014s] ()
2022-04-06 23:01:53,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 10ef925b9a32 -> 3a1141b115bc, insert cart_goods table
2022-04-06 23:01:53,1

In [306]:
class CartGoods(Entity):
    def __init__(self):
        super().__init__()
        self.cart_id = -1
        self.goods_id = -1

#### 建立goods

In [307]:
!alembic revision -m "create goods table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/67
  9bc167d9a6_insert_goods_table.py ...  done


In [308]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 23:07:33,161 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 23:07:33,161 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:07:33,163 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 23:07:33,163 INFO sqlalchemy.engine.Engine [generated in 0.00015s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00015s] ()
2022-04-06 23:07:33,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 3a1141b115bc -> 679bc167d9a6, insert goods table
2022-04-06 23:07:33,173 IN

In [309]:
class Goods(Entity):
    def __init__(self):
        super().__init__()
        self.product_id = -1
        self.amount = -1

#### 建立product

In [311]:
!alembic revision -m "create product table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/05
  171e77f55d_create_product_table.py ...  done


In [312]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 23:14:26,810 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 23:14:26,810 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:14:26,812 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 23:14:26,812 INFO sqlalchemy.engine.Engine [generated in 0.00017s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00017s] ()
2022-04-06 23:14:26,818 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 679bc167d9a6 -> 05171e77f55d, create product table
2022-04-06 23:14:26,820 

In [313]:
class Product(Entity):
    def __init__(self):
        super().__init__()
        self.name = ''
        self.manufacturer = ''
        self.origin = ''
        self.production_date = None
        self.shelf_life = ''
        self.desc = ''
        self.category_id = -1    

#### 建立category

In [314]:
!alembic revision -m "create category table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/4b
  a3059ff18c_create_category_table.py ...  done


In [315]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-06 23:18:30,864 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-06 23:18:30,864 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-06 23:18:30,866 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-06 23:18:30,866 INFO sqlalchemy.engine.Engine [generated in 0.00025s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00025s] ()
2022-04-06 23:18:30,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 05171e77f55d -> 4ba3059ff18c, create category table
2022-04-06 23:18:30,874

In [316]:
class Category(Entity):
    def __init__(self):
        super().__init__()
        self.name = ''

#### 建立vendor_district

In [399]:
!alembic revision -m "create vendor_district table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/49
  2d91d5eb58_create_vendor_district_table.py ...  done


In [ ]:
!alembic upgrade head

In [400]:
class VendorDistrict(Entity):
    def __init__(self):
        super().__init__()   
        self.vendor_id = -1
        self.district_id = -1

#### 改变vendor，添加来源

In [48]:
!alembic revision -m "alter vendor add origin table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/6e
  af45c03ca1_alter_vendor_add_origin_table.py ...  done


In [49]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-07 12:32:15,187 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-07 12:32:15,187 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-07 12:32:15,189 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-07 12:32:15,189 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-07 12:32:15,201 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 492d91d5eb58 -> 6eaf45c03ca1, alter vendor add origin table
2022-04-07 12:3

#### 改变vendor，添加下单时间

In [76]:
!alembic revision -m "alter vendor add available time table" 

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/e3
  3134e477eb_alter_vendor_add_available_time_table.py ...  done


In [77]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-07 14:10:52,457 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-07 14:10:52,457 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-07 14:10:52,458 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-07 14:10:52,458 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-07 14:10:52,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 6eaf45c03ca1 -> e33134e477eb, alter vendor add available time table
2022-04

#### 改变vendor，添加出入源

In [332]:
!alembic revision -m "alter vendor add input table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/d8
  ffe352714a_alter_vendor_add_input_table.py ...  done


In [327]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-10 19:58:08,028 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-10 19:58:08,028 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-10 19:58:08,029 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-10 19:58:08,029 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()


### 重置Scheme

In [350]:
def reset_db():
    db.drop('alembic_version')
    db.drop('district')
    db.drop('vendor')
    db.drop('vendor_tag')
    db.drop('tag')
    db.drop('cart')
    db.drop('cart_goods')
    db.drop('goods')
    db.drop('product')
    db.drop('category')
    db.drop('vendor_district')

In [351]:
reset_db()
db.query('SELECT name FROM sqlite_master WHERE type="table"')

DEBUG 2022-04-10 23:02:47,241 [<ipython-input-90-490d81572b32>:60]: drop table if exists alembic_version
DEBUG 2022-04-10 23:02:47,243 [<ipython-input-90-490d81572b32>:60]: drop table if exists district
DEBUG 2022-04-10 23:02:47,245 [<ipython-input-90-490d81572b32>:60]: drop table if exists vendor
DEBUG 2022-04-10 23:02:47,248 [<ipython-input-90-490d81572b32>:60]: drop table if exists vendor_tag
DEBUG 2022-04-10 23:02:47,249 [<ipython-input-90-490d81572b32>:60]: drop table if exists tag
DEBUG 2022-04-10 23:02:47,251 [<ipython-input-90-490d81572b32>:60]: drop table if exists cart
DEBUG 2022-04-10 23:02:47,253 [<ipython-input-90-490d81572b32>:60]: drop table if exists cart_goods
DEBUG 2022-04-10 23:02:47,255 [<ipython-input-90-490d81572b32>:60]: drop table if exists goods
DEBUG 2022-04-10 23:02:47,256 [<ipython-input-90-490d81572b32>:60]: drop table if exists product
DEBUG 2022-04-10 23:02:47,258 [<ipython-input-90-490d81572b32>:60]: drop table if exists category
DEBUG 2022-04-10 23:02:4

[('test',)]

In [352]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-10 23:02:48,548 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-10 23:02:48,548 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-10 23:02:48,549 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA temp.table_info("alembic_version")
2022-04-10 23:02:48,549 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-10 23:02:48,550 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
2022-04-10 23:02:48,550 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-10 2

In [404]:
db.query('SELECT name FROM sqlite_master WHERE type="table"')

[('test',),
 ('alembic_version',),
 ('district',),
 ('vendor',),
 ('cart',),
 ('tag',),
 ('vendor_tag',),
 ('cart_goods',),
 ('goods',),
 ('product',),
 ('category',),
 ('vendor_district',)]

### 初始化district

In [405]:
dataset = db.query('select * from district')
ditricts = []
for data in dataset:
    ditrict = District()
    ditrict.id, ditrict.name, ditrict.code, ditrict.created_at, ditrict.updated_at, ditrict.state  = data
    ditricts.append(ditrict)
ditricts

[{'id': 1, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '上海市', 'code': '31'},
 {'id': 2, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '黄浦区', 'code': '310101'},
 {'id': 3, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '徐汇区', 'code': '310104'},
 {'id': 4, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '长宁区', 'code': '310105'},
 {'id': 5, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '静安区', 'code': '310106'},
 {'id': 6, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '普陀区', 'code': '310107'},
 {'id': 7, 'created_at': '2022-04-07 00:27:30.463253', 'updated_at': '2022-04-07 00:27:30.463253', 'state': 1, 'name': '虹口区', 'code': 

### Excel ETL

In [64]:
import csv
def load_csv(csvpath):
    fd = open(csvpath)
    reader = csv.reader(fd)
    data = list(map(lambda row: row, reader))
    dataframe = {}
    for i, r in enumerate(data):
        dataframe[i] = {}
        row = data[i]
        for j, c in enumerate(row):
            dataframe[i][j] = c
    return pd.DataFrame(dataframe)

## 0000001 CSV ETL

In [102]:
root = os.path.join('data', 'input', '0000001')
csv_001 = os.path.join(root, '001.csv')
csv_002 = os.path.join(root, '002.csv')

### 001 

In [103]:
csv_001_dataframes = load_csv(csv_001)
csv_001_dataframes.T

,0,1,2,3,4
0,序号,渠道,类型,下单时间,特点
1,1,盒马,APP,7:30或8:00\n（不同区门店时间可能不同，请参见盒马APP首页通知）,APP抢菜情况视门店覆盖范围而定，竞争性各不相同，但抢菜攻略大致相同。以下已盒马APP成功案...
2,2,叮咚买菜,APP,6:00或8:30,
3,3,每日优鲜,APP,8:00,
4,4,美团买菜,APP,6:00,
5,5,美团外卖,APP,全天,1.美团外卖找到美团买菜入口，选好自己的菜，然后再便利店或者其他外卖产品选个订单，美团外卖和...
6,6,京东到家,APP/小程序,可以随时下单,集成很多超市，商品供货情况依赖于超市库存，不同门店运费不同
7,7,大润发优鲜,APP/小程序,22:00,可预约第二天菜品
8,8,光明随心订,小程序,可以随时下单,可以订购牛奶，购买蔬菜包
9,9,永辉生活,APP/小程序,可以随时下单,生鲜食品有售，但依赖于小区所在地点附近是否有门店


In [81]:
dataframe = csv_001_dataframes[1]
vendor = Vendor()
vendor.name = dataframe[1]
vendor.origin = dataframe[2]
vendor.available_desc = dataframe[2]
vendor.desc = dataframe[4]
vendor

{'id': -1, 'created_at': None, 'updated_at': None, 'state': 1, 'name': '盒马', 'contact': '', 'address': '', 'mobile': '', 'tel': '', 'desc': 'APP抢菜情况视门店覆盖范围而定，竞争性各不相同，但抢菜攻略大致相同。以下已盒马APP成功案例为例：\n1.提前一天盘点家中需够货品，进入盒马APP，提前加入购物车，此时可能有部分生鲜无货\n2.购买当天，按照下单时间提前10分钟进入盒马APP（如所需货品较多，可提前15-30分钟），将前一日无货生鲜加入购物车，加完之后停留在购物车界面\n3.等到下单时间，持续点击结算，如遇前方拥堵提醒或无配送小哥界面，返回结算界面继续点击结算，一般情况下5-10分钟后，可提交订单，5分钟后没有出现提交订单也不要着急，继续点击结算。\n4.提交订单时，需选择配送时间，时间尽量往中午之后选（较早的时间可能导致拥堵）\n5.可不定时刷新，据了解，早上8点左右、中午12点左右、早上6点均有网友有购物记录', 'wechat': '', 'origin': 'APP', 'available_desc': 'APP'}

In [16]:
db.find('vendor')

[(1,
  '盒马',
  '',
  '',
  '',
  '',
  'APP抢菜情况视门店覆盖范围而定，竞争性各不相同，但抢菜攻略大致相同。以下已盒马APP成功案例为例：\n1.提前一天盘点家中需够货品，进入盒马APP，提前加入购物车，此时可能有部分生鲜无货\n2.购买当天，按照下单时间提前10分钟进入盒马APP（如所需货品较多，可提前15-30分钟），将前一日无货生鲜加入购物车，加完之后停留在购物车界面\n3.等到下单时间，持续点击结算，如遇前方拥堵提醒或无配送小哥界面，返回结算界面继续点击结算，一般情况下5-10分钟后，可提交订单，5分钟后没有出现提交订单也不要着急，继续点击结算。\n4.提交订单时，需选择配送时间，时间尽量往中午之后选（较早的时间可能导致拥堵）\n5.可不定时刷新，据了解，早上8点左右、中午12点左右、早上6点均有网友有购物记录',
  '',
  '2022-04-14 20:07:55.582069',
  '2022-04-14 20:07:55.582069',
  1,
  'APP',
  '7:30或8:00\n（不同区门店时间可能不同，请参见盒马APP首页通知）',
  None,
  '0000001/001',
  'csv',
  ''),
 (2,
  '叮咚买菜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-14 20:07:55.584008',
  '2022-04-14 20:07:55.584008',
  1,
  'APP',
  '6:00或8:30',
  None,
  '0000001/001',
  'csv',
  ''),
 (3,
  '每日优鲜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-14 20:07:55.584851',
  '2022-04-14 20:07:55.584851',
  1,
  'APP',
  '8:00',
  None,
  '0000001/001',
  'csv',
  ''),
 (4,
  '美团买菜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-14 20:07:55.58563

### 002

In [104]:
csv_002_dataframes = load_csv(csv_002)
csv_002_dataframes.T

,0,1,2,3,4,5,6
0,序号,来源出处,名称,团购起送份数,配送范围,联系人,配送内容请参见附件团购详情页
1,1,浦东发布,上海乐得农产品专业合作社,30只起送,,杭女士13818949339,
2,2,浦东发布,上海联庆农业发展专业合作联社,20份起送,,丁玲19921594920,
3,3,浦东发布,上海田博瓜果专业合作社,30份起送,,刘仁13818818069,
4,4,浦东发布,上海团金农业投资发展（集团）有限公司,50份起送,,宋雪鹤13621983969,
5,5,浦东发布,上海卉绿蔬菜生态园艺场,100份起送,,徐浩18602164931,
6,6,浦东发布,上海绿妮瓜果专业合作社,30份起送,,朱蓓13166448213,
7,7,浦东发布,米果时鲜,30份起送,,郭中林15221708940,
8,8,浦东发布,上海绿艳农家乐专业合作社,满2000元起送,,凤飞荣13601601895,
9,9,浦东发布,上海沐旭果蔬专业合作社,20份起送,,潘先生13611641318,


In [106]:
dataframe = csv_002_dataframes[1]
vendor = Vendor()
vendor.name = dataframe[2]
vendor.origin = dataframe[1]
vendor.desc = dataframe[3]
vendor.available_desc = dataframe[4]
vendor.contact = dataframe[5]
vendor.mobile = dataframe[5]
vendor

{'id': -1, 'created_at': None, 'updated_at': None, 'state': 1, 'name': '上海乐得农产品专业合作社', 'contact': '杭女士13818949339', 'address': '', 'mobile': '杭女士13818949339', 'tel': '', 'desc': '30只起送', 'wechat': '', 'origin': '浦东发布', 'available_desc': ''}

In [152]:
db.delete('vendor')

DEBUG 2022-04-07 23:43:05,418 [<ipython-input-90-490d81572b32>:60]: delete from vendor


In [154]:
db.find('vendor')

[(1,
  '盒马',
  '',
  '',
  '',
  '',
  'APP抢菜情况视门店覆盖范围而定，竞争性各不相同，但抢菜攻略大致相同。以下已盒马APP成功案例为例：\n1.提前一天盘点家中需够货品，进入盒马APP，提前加入购物车，此时可能有部分生鲜无货\n2.购买当天，按照下单时间提前10分钟进入盒马APP（如所需货品较多，可提前15-30分钟），将前一日无货生鲜加入购物车，加完之后停留在购物车界面\n3.等到下单时间，持续点击结算，如遇前方拥堵提醒或无配送小哥界面，返回结算界面继续点击结算，一般情况下5-10分钟后，可提交订单，5分钟后没有出现提交订单也不要着急，继续点击结算。\n4.提交订单时，需选择配送时间，时间尽量往中午之后选（较早的时间可能导致拥堵）\n5.可不定时刷新，据了解，早上8点左右、中午12点左右、早上6点均有网友有购物记录',
  '',
  '2022-04-07 23:43:08.515112',
  '2022-04-07 23:43:08.515112',
  1,
  'APP',
  'APP',
  None,
  '0000001/001',
  'csv'),
 (2,
  '叮咚买菜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-07 23:43:08.516896',
  '2022-04-07 23:43:08.516896',
  1,
  'APP',
  'APP',
  None,
  '0000001/001',
  'csv'),
 (3,
  '每日优鲜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-07 23:43:08.518115',
  '2022-04-07 23:43:08.518115',
  1,
  'APP',
  'APP',
  None,
  '0000001/001',
  'csv'),
 (4,
  '美团买菜',
  '',
  '',
  '',
  '',
  '',
  '',
  '2022-04-07 23:43:08.519277',
  '2022-04-07 23:43:08.519277',
  1,
  'APP',
  'APP',


#### 抽取联系人

In [114]:
import re
def match_chinese(str):
    return re.findall(r'[\u4e00-\u9fa5]+', str)

In [146]:
m = match_chinese('朱蓓13166448213ss')
m

['朱蓓']

#### 抽取手机

In [147]:
def match_mobile(str):
    return re.findall(r'1[358]\d+', str)

In [148]:
m = match_mobile('朱蓓13166448213ss')
m

['13166448213']

#### 抽取手机

In [139]:
def match_tel(str):
    return re.findall(r'\(?0\d{2,3}[)-]?\d{7,8}', str)

In [140]:
m = match_tel('朱蓓65487222ss')
m

[]

## 生成报告

#### 模板

In [216]:
report_html_rpl = '''
<table>
<!-- {{ for i, vendor in enumerate(vendors) 
<tr>
    <td align="left" valign="bottom" class="heading"><!-- {{ '{}'.format(i) }} --></td>
    <td align="right" valign="bottom" class="district"><b><!-- {{ '{}'.format(vendor.district) }} --></b></td>
    <td align="right" valign="bottom" class="vendor"><b><!-- {{ '{}'.format(vendor.name) }} --></b></td>
    <td align="right" valign="bottom" class="contact"><b><!-- {{ '{}'.format(vendor.contact) }} --></b></td>
    <td align="right" valign="bottom" class="address"><b><!-- {{ '{}'.format(vendor.address) }} --></b></td>
    <td align="right" valign="bottom" class="mobile"><b><!-- {{ '{}'.format(vendor.mobile) }} --></b></td>
    <td align="right" valign="bottom" class="category"><b><!-- {{ '{}'.format(vendor.category) }} --></b></td>
    <td align="right" valign="bottom" class="desc"><b><!-- {{ '{}'.format(vendor.desc) }} --></b></td>
    <td align="right" valign="bottom" class="origin"><b><!-- {{ '{}'.format(vendor.origin) }} --></b></td>
    <td align="right" valign="bottom" class="order_type"><b><!-- {{ '{}'.format(vendor.order_type) }} --></b></td>
    <td align="right" valign="bottom" class="available_date"><b><!-- {{ '{}'.format(vendor.available_date) }} --></b></td>
    <td align="right" valign="bottom" class="record_date"><b><!-- {{ '{}'.format(vendor.record_date) }} --></b></td>
    <td align="right" valign="bottom" class="tag"><b><!-- {{ '{}'.format(vendor.tag) }} --></b></td>
</tr>
}} -->
<table>
'''
report_html_rpl

'\n<table>\n<!-- {{ for i, vendor in enumerate(vendors) \n<tr>\n    <td align="left" valign="bottom" class="heading"><!-- {{ \'{}\'.format(i) }} --></td>\n    <td align="right" valign="bottom" class="district"><b><!-- {{ \'{}\'.format(vendor.district) }} --></b></td>\n    <td align="right" valign="bottom" class="vendor"><b><!-- {{ \'{}\'.format(vendor.name) }} --></b></td>\n    <td align="right" valign="bottom" class="contact"><b><!-- {{ \'{}\'.format(vendor.contact) }} --></b></td>\n    <td align="right" valign="bottom" class="address"><b><!-- {{ \'{}\'.format(vendor.address) }} --></b></td>\n    <td align="right" valign="bottom" class="mobile"><b><!-- {{ \'{}\'.format(vendor.mobile) }} --></b></td>\n    <td align="right" valign="bottom" class="category"><b><!-- {{ \'{}\'.format(vendor.category) }} --></b></td>\n    <td align="right" valign="bottom" class="desc"><b><!-- {{ \'{}\'.format(vendor.desc) }} --></b></td>\n    <td align="right" valign="bottom" class="origin"><b><!-- {{ \'{}\

<table>
<tr>
            <td align="left" valign="bottom" class="heading">
0
</td>
            <td align="right" valign="bottom" class="district"><b>
mcpuwfeyls
</b></td>
            <td align="right" valign="bottom" class="vendor"><b>
qaliyj
</b></td>
            <td align="right" valign="bottom" class="contact"><b>
yimgzy
</b></td>
            <td align="right" valign="bottom" class="address"><b>
sdvk
</b></td>
            <td align="right" valign="bottom" class="mobile"><b>
wiikqiqv
</b></td>
            <td align="right" valign="bottom" class="category"><b>
wmoiancz
</b></td>
            <td align="right" valign="bottom" class="desc"><b>
paxqzun
</b></td>
            <td align="right" valign="bottom" class="origin"><b>

</b></td>
            <td align="right" valign="bottom" class="order_type"><b>

</b></td>
            <td align="right" valign="bottom" class="available_date"><b>

</b></td>
            <td align="right" valign="bottom" class="record_date"><b>

</b></td>
            <td align="right" valign="bottom" class="tag"><b>

</b></td>
        </tr>
        <tr>
            <td align="left" valign="bottom" class="heading">
1
</td>
            <td align="right" valign="bottom" class="district"><b>
rweqbq
</b></td>
            <td align="right" valign="bottom" class="vendor"><b>
yylndqci
</b></td>
            <td align="right" valign="bottom" class="contact"><b>
lim
</b></td>
            <td align="right" valign="bottom" class="address"><b>
wfotsh
</b></td>
            <td align="right" valign="bottom" class="mobile"><b>
uguuwhcw
</b></td>
            <td align="right" valign="bottom" class="category"><b>
ocnyoe
</b></td>
            <td align="right" valign="bottom" class="desc"><b>
ijj
</b></td>
            <td align="right" valign="bottom" class="origin"><b>

</b></td>
            <td align="right" valign="bottom" class="order_type"><b>

</b></td>
            <td align="right" valign="bottom" class="available_date"><b>

</b></td>
            <td align="right" valign="bottom" class="record_date"><b>

</b></td>
            <td align="right" valign="bottom" class="tag"><b>

</b></td>
        </tr>
        <tr>
            <td align="left" valign="bottom" class="heading">
2
</td>
            <td align="right" valign="bottom" class="district"><b>
gmjpp
</b></td>
            <td align="right" valign="bottom" class="vendor"><b>
pxdkgfsezg
</b></td>
            <td align="right" valign="bottom" class="contact"><b>
uos
</b></td>
            <td align="right" valign="bottom" class="address"><b>
czb
</b></td>
            <td align="right" valign="bottom" class="mobile"><b>
aejsgsz
</b></td>
            <td align="right" valign="bottom" class="category"><b>
mbwt
</b></td>
            <td align="right" valign="bottom" class="desc"><b>
pwfrbkggfd
</b></td>
            <td align="right" valign="bottom" class="origin"><b>

</b></td>
            <td align="right" valign="bottom" class="order_type"><b>

</b></td>
            <td align="right" valign="bottom" class="available_date"><b>

</b></td>
            <td align="right" valign="bottom" class="record_date"><b>

</b></td>
            <td align="right" valign="bottom" class="tag"><b>

</b></td>
        </tr>
        <tr>
            <td align="left" valign="bottom" class="heading">
3
</td>
            <td align="right" valign="bottom" class="district"><b>
aed
</b></td>
            <td align="right" valign="bottom" class="vendor"><b>
tmrek
</b></td>
            <td align="right" valign="bottom" class="contact"><b>
djijnskoi
</b></td>
            <td align="right" valign="bottom" class="address"><b>
dknctw
</b></td>
            <td align="right" valign="bottom" class="mobile"><b>
iun
</b></td>
            <td align="right" valign="bottom" class="category"><b>
nktvbleau
</b></td>
            <td align="right" valign="bottom" class="desc"><b>
xglvyxragp
</b></td>
            <td align="right" valign="bottom" class="origin"><b>

</b></td>
            <td align="right" valign="bottom" class="order_type"><b>

</b></td>
            <td align="right" valign="bottom" class="available_date"><b>

</b></td>
            <td align="right" valign="bottom" class="record_date"><b>

</b></td>
            <td align="right" valign="bottom" class="tag"><b>

</b></td>
        </tr>
        <tr>
            <td align="left" valign="bottom" class="heading">
4
</td>
            <td align="right" valign="bottom" class="district"><b>
myuygs
</b></td>
            <td align="right" valign="bottom" class="vendor"><b>
regjwhd
</b></td>
            <td align="right" valign="bottom" class="contact"><b>
jesnnt
</b></td>
            <td align="right" valign="bottom" class="address"><b>
ssnnvp
</b></td>
            <td align="right" valign="bottom" class="mobile"><b>
cpt
</b></td>
            <td align="right" valign="bottom" class="category"><b>
kpym
</b></td>
            <td align="right" valign="bottom" class="desc"><b>
ujdigooxty
</b></td>
            <td align="right" valign="bottom" class="origin"><b>

</b></td>
            <td align="right" valign="bottom" class="order_type"><b>

</b></td>
            <td align="right" valign="bottom" class="available_date"><b>

</b></td>
            <td align="right" valign="bottom" class="record_date"><b>

</b></td>
            <td align="right" valign="bottom" class="tag"><b>

</b></td>
        </tr>
</table>

## ETL工厂

In [233]:
!python3 etl_factory.py

INFO 2022-04-09 13:33:27,653 [header.py:166]: -*-*-*-*-*-*-*-*-*-*-* ETLFactoryUnitTests:build_test begin -*-*-*-*-*-*-*-*-*-*-*
INFO 2022-04-09 13:33:27,653 [etl.py:13]: ----------------------- input_0000001_csv_etl ETL start -----------------------
INFO 2022-04-09 13:33:27,653 [etl.py:15]: ---- input_0000001_csv_etl extract start ----
INFO 2022-04-09 13:33:27,659 [etl.py:18]: ---- input_0000001_csv_etl extract end elapse 0.005 seconds ----
INFO 2022-04-09 13:33:27,659 [etl.py:19]: ---- input_0000001_csv_etl transfer start ----
INFO 2022-04-09 13:33:27,661 [etl.py:22]: ---- input_0000001_csv_etl transfer end elapse 0.003 seconds ----
INFO 2022-04-09 13:33:27,661 [etl.py:23]: ---- input_0000001_csv_etl load start ----
DEBUG 2022-04-09 13:33:27,661 [db.py:70]: ('insert into vendor(created_at,updated_at,state,name,contact,address,mobile,tel,desc,wechat,origin,available_desc,input_path,input_type) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?)', [datetime.datetime(2022, 4, 9, 13, 33, 27, 661927), da

In [234]:
len(db.find('vendor'))

120

## 得到所有vendorview list

In [236]:
class VendorView:
    def __init__(self, **kwargs):
        self.district = kwargs['district'] if 'district' in kwargs else ''
        self.name = kwargs['name'] if 'name' in kwargs else ''
        self.contact = kwargs['contact'] if 'contact' in kwargs else ''
        self.address = kwargs['address'] if 'address' in kwargs else ''
        self.mobile = kwargs['mobile'] if 'mobile' in kwargs else ''
        self.category = kwargs['category'] if 'category' in kwargs else ''
        self.desc = kwargs['desc'] if 'desc' in kwargs else ''
        self.origin = kwargs['district'] if 'origin' in kwargs else ''
        self.order_type = kwargs['order_type'] if 'order_type' in kwargs else ''
        self.available_date = kwargs['available_date'] if 'available_date' in kwargs else ''
        self.record_date = kwargs['record_date'] if 'record_date' in kwargs else ''
        self.tag = kwargs['tag'] if 'tag' in kwargs else ''

    def __repr__(self):
        return str(self.__dict__)

In [298]:
!python3 db.py

INFO 2022-04-10 16:29:25,900 [header.py:265]: -*-*-*-*-*-*-*-*-*-*-* DBUnitTests:all_entities_test skipped -*-*-*-*-*-*-*-*-*-*-*
INFO 2022-04-10 16:29:25,900 [header.py:265]: -*-*-*-*-*-*-*-*-*-*-* DBUnitTests:entity_fields_test skipped -*-*-*-*-*-*-*-*-*-*-*
INFO 2022-04-10 16:29:25,900 [header.py:268]: -*-*-*-*-*-*-*-*-*-*-* DBUnitTests:get_all_vendors_test begin -*-*-*-*-*-*-*-*-*-*-*
DEBUG 2022-04-10 16:29:25,900 [db.py:102]: ('select * from vendor', [])
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,904 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,905 [db.py:102]: PRAGMA table_info = 'vendor'
DEBUG 2022-04-10 16:29:25,905 [d

## 主应用

In [21]:
!python3 main.py

DEBUG 2022-04-16 00:41:11,785 [db.py:137]: select * from district
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,786 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRAGMA table_info = 'district'
DEBUG 2022-04-16 00:41:11,787 [db.py:137]: PRA

## 改变vendor，添加 原始链接

In [306]:
!alembic revision -m "alter vendor add link table"

  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/versions/bf
  b12a35d1bd_alter_vendor_add_link_table.py ...  done


In [307]:
!alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-10 16:37:03,162 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-10 16:37:03,162 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-10 16:37:03,166 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num 
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num 
FROM alembic_version
2022-04-10 16:37:03,166 INFO sqlalchemy.engine.Engine [generated in 0.00029s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00029s] ()
2022-04-10 16:37:03,178 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [alembic.runtime.migration] Running upgrade 0ad5adfdd8c2 -> 872e7deeb339, alter vendor add link table
2022-04-10 16:37:

In [357]:
db.query('select * from district')

[(1,
  '上海市',
  '31',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (2,
  '黄浦区',
  '310101',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (3,
  '徐汇区',
  '310104',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (4,
  '长宁区',
  '310105',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (5,
  '静安区',
  '310106',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (6,
  '普陀区',
  '310107',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (7,
  '虹口区',
  '310109',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (8,
  '杨浦区',
  '310110',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (9,
  '闵行区',
  '310112',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (10,
  '宝山区',
  '310113',
  '2022-04-10 23:02:48.581206',
  '2022-04-10 23:02:48.581206',
  1),
 (11,
  '嘉定区',
  '310114',
  '2022-04-10 23

In [356]:
db.query('select * from vendor_district')

[(1,
  39,
  '310115',
  '2022-04-10 23:13:42.197384',
  '2022-04-10 23:13:42.197384',
  1),
 (2,
  40,
  '310115',
  '2022-04-10 23:13:42.201146',
  '2022-04-10 23:13:42.201146',
  1),
 (3,
  41,
  '310115',
  '2022-04-10 23:13:42.204470',
  '2022-04-10 23:13:42.204470',
  1),
 (4,
  42,
  '310115',
  '2022-04-10 23:13:42.208970',
  '2022-04-10 23:13:42.208970',
  1),
 (5,
  43,
  '310115',
  '2022-04-10 23:13:42.214150',
  '2022-04-10 23:13:42.214150',
  1),
 (6,
  44,
  '310115',
  '2022-04-10 23:13:42.216957',
  '2022-04-10 23:13:42.216957',
  1),
 (7,
  45,
  '310115',
  '2022-04-10 23:13:42.219877',
  '2022-04-10 23:13:42.219877',
  1),
 (8,
  46,
  '310115',
  '2022-04-10 23:13:42.222512',
  '2022-04-10 23:13:42.222512',
  1),
 (9,
  47,
  '310115',
  '2022-04-10 23:13:42.225023',
  '2022-04-10 23:13:42.225023',
  1),
 (10,
  48,
  '310115',
  '2022-04-10 23:13:42.227244',
  '2022-04-10 23:13:42.227244',
  1),
 (11,
  49,
  '310115',
  '2022-04-10 23:13:42.229451',
  '2022-04-10